# Gesture Recognition - Model 5
#### Conv3D Model with comprehensive preprocessing (using OpenCV library - Resize and Data augmetation with Affine transformation, Horizontal Flip, Cropping and Resizing,  Usage of Optimizer, ReduceLROnPlateau to reduce learning rate and experimenting with various sizes of batch sizes and Epochs )
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime
import os
import cv2

Set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator and Preprocessing functions
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
# Function to initialize all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [7]:
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = initialize_batch_data(batch_size)
    
    # For building an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # For building an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Crop non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [8]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)          
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 5

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

nb_classes=5

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
model.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(16, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 10, 40, 40, 32)    13856     
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 223s 3s/step - loss: 2.0028 - categorical_accuracy: 0.3017 - val_loss: 3.6736 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-1.99279-0.30367-3.67358-0.20000.h5
Epoch 2/20
67/67 [==============================] - 65s 964ms/step - loss: 1.9123 - categorical_accuracy: 0.3167 - val_loss: 3.6978 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-1.91228-0.31675-3.69783-0.39000.h5
Epoch 3/20
67/67 [==============================] - 64s 962ms/step - loss: 1.5517 - categorical_accuracy: 0.4046 - val_loss: 1.1850 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-1.55171-0.40464-1.18503-0.55000.h5
Epoch 4/20
67/67 [==============================] - 6

In [16]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 33s 971ms/step - loss: 0.6046 - categorical_accuracy: 0.7516 - val_loss: 0.5299 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-0.60463-0.75163-0.52989-0.80000.h5
Epoch 2/20
34/34 [==============================] - 32s 952ms/step - loss: 0.7756 - categorical_accuracy: 0.6797 - val_loss: 0.5599 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-0.77565-0.67974-0.55992-0.76000.h5
Epoch 3/20
34/34 [==============================] - 33s 956ms/step - loss: 0.6870 - categorical_accuracy: 0.7190 - val_loss: 0.4504 - val_categorical_accuracy: 0.8200

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-0.68699-0.71895-0.45045-0.82000.h5
Epoch 4/20
34/34 [==============================] - 32s 955ms/step - loss: 0.6505 - categorical_accuracy: 0.7549 - val_loss: 0.5493 - val_categorical_accurac

In [17]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 23s 982ms/step - loss: 0.7801 - categorical_accuracy: 0.7005 - val_loss: 0.3135 - val_categorical_accuracy: 0.9500

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-0.78009-0.70048-0.31349-0.95000.h5
Epoch 2/30
23/23 [==============================] - 22s 960ms/step - loss: 0.8949 - categorical_accuracy: 0.6522 - val_loss: 0.3885 - val_categorical_accuracy: 0.8500

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-0.89493-0.65217-0.38848-0.85000.h5
Epoch 3/30
23/23 [==============================] - 22s 963ms/step - loss: 0.5514 - categorical_accuracy: 0.7971 - val_loss: 0.6634 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-0.55136-0.79710-0.66337-0.80000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.
Epoch 4/30
23/23 [==============================] - 22s 962ms/step - loss: 0

23/23 [==============================] - 22s 964ms/step - loss: 0.8166 - categorical_accuracy: 0.6715 - val_loss: 0.6532 - val_categorical_accuracy: 0.7250

Epoch 00027: saving model to model_init_2020-03-3013_25_59.182049/model-00027-0.81660-0.67150-0.65316-0.72500.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.1920929521291868e-10.
Epoch 28/30
23/23 [==============================] - 22s 963ms/step - loss: 0.5959 - categorical_accuracy: 0.7633 - val_loss: 0.3132 - val_categorical_accuracy: 0.9500

Epoch 00028: saving model to model_init_2020-03-3013_25_59.182049/model-00028-0.59590-0.76329-0.31321-0.95000.h5
Epoch 29/30
23/23 [==============================] - 22s 964ms/step - loss: 0.6076 - categorical_accuracy: 0.7536 - val_loss: 0.4336 - val_categorical_accuracy: 0.8250

Epoch 00029: saving model to model_init_2020-03-3013_25_59.182049/model-00029-0.60758-0.75362-0.43356-0.82500.h5
Epoch 30/30
23/23 [==============================] - 22s 960ms/step - loss: 0.6197 -

In [18]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 14s 989ms/step - loss: 0.4123 - categorical_accuracy: 0.8492 - val_loss: 0.3884 - val_categorical_accuracy: 0.9500

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-0.41235-0.84921-0.38841-0.95000.h5
Epoch 2/30
14/14 [==============================] - 13s 958ms/step - loss: 0.6022 - categorical_accuracy: 0.7619 - val_loss: 0.3838 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-0.60221-0.76190-0.38385-0.80000.h5
Epoch 3/30
14/14 [==============================] - 13s 953ms/step - loss: 0.5822 - categorical_accuracy: 0.7857 - val_loss: 0.6762 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-0.58223-0.78571-0.67618-0.75000.h5
Epoch 4/30
14/14 [==============================] - 13s 951ms/step - loss: 0.6520 - categorical_accuracy: 0.7302 - val_loss: 0.3751 - val_categorical_accurac

14/14 [==============================] - 13s 950ms/step - loss: 0.6760 - categorical_accuracy: 0.7222 - val_loss: 0.6032 - val_categorical_accuracy: 0.7500

Epoch 00028: saving model to model_init_2020-03-3013_25_59.182049/model-00028-0.67603-0.72222-0.60317-0.75000.h5
Epoch 29/30
14/14 [==============================] - 13s 948ms/step - loss: 0.5582 - categorical_accuracy: 0.7698 - val_loss: 0.3506 - val_categorical_accuracy: 0.9000

Epoch 00029: saving model to model_init_2020-03-3013_25_59.182049/model-00029-0.55823-0.76984-0.35064-0.90000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.164153273563659e-13.
Epoch 30/30
14/14 [==============================] - 13s 956ms/step - loss: 0.7063 - categorical_accuracy: 0.6984 - val_loss: 0.4869 - val_categorical_accuracy: 0.8500

Epoch 00030: saving model to model_init_2020-03-3013_25_59.182049/model-00030-0.70635-0.69841-0.48690-0.85000.h5


In [19]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 8s 1s/step - loss: 0.6459 - categorical_accuracy: 0.7361 - val_loss: 0.5438 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-0.64595-0.73611-0.54379-0.85000.h5
Epoch 2/30
8/8 [==============================] - 8s 993ms/step - loss: 0.6482 - categorical_accuracy: 0.7083 - val_loss: 0.7323 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-0.64817-0.70833-0.73233-0.75000.h5
Epoch 3/30
8/8 [==============================] - 8s 990ms/step - loss: 0.7125 - categorical_accuracy: 0.7222 - val_loss: 0.4879 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-0.71254-0.72222-0.48795-0.80000.h5
Epoch 4/30
8/8 [==============================] - 8s 991ms/step - loss: 0.8119 - categorical_accuracy: 0.7083 - val_loss: 0.6367 - val_categorical_accuracy: 0.8000

Epoc


Epoch 00027: saving model to model_init_2020-03-3013_25_59.182049/model-00027-0.52244-0.84722-0.46397-0.75000.h5
Epoch 28/30
8/8 [==============================] - 8s 982ms/step - loss: 0.3506 - categorical_accuracy: 0.9028 - val_loss: 0.5035 - val_categorical_accuracy: 0.8500

Epoch 00028: saving model to model_init_2020-03-3013_25_59.182049/model-00028-0.35057-0.90278-0.50349-0.85000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 5.684342156072553e-17.
Epoch 29/30
8/8 [==============================] - 8s 987ms/step - loss: 0.7315 - categorical_accuracy: 0.6944 - val_loss: 0.4200 - val_categorical_accuracy: 0.9500

Epoch 00029: saving model to model_init_2020-03-3013_25_59.182049/model-00029-0.73154-0.69444-0.42000-0.95000.h5
Epoch 30/30
8/8 [==============================] - 8s 992ms/step - loss: 0.7182 - categorical_accuracy: 0.6944 - val_loss: 0.4014 - val_categorical_accuracy: 0.9000

Epoch 00030: saving model to model_init_2020-03-3013_25_59.182049/model-00030-0.7

In [20]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 8s 1s/step - loss: 0.7090 - categorical_accuracy: 0.7500 - val_loss: 0.4534 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-03-3013_25_59.182049/model-00001-0.70904-0.75000-0.45345-0.85000.h5
Epoch 2/20
8/8 [==============================] - 8s 994ms/step - loss: 0.7087 - categorical_accuracy: 0.7639 - val_loss: 0.2832 - val_categorical_accuracy: 0.9500

Epoch 00002: saving model to model_init_2020-03-3013_25_59.182049/model-00002-0.70869-0.76389-0.28323-0.95000.h5
Epoch 3/20
8/8 [==============================] - 8s 990ms/step - loss: 0.6320 - categorical_accuracy: 0.8056 - val_loss: 0.3387 - val_categorical_accuracy: 0.9000

Epoch 00003: saving model to model_init_2020-03-3013_25_59.182049/model-00003-0.63200-0.80556-0.33875-0.90000.h5
Epoch 4/20
8/8 [==============================] - 8s 984ms/step - loss: 0.6677 - categorical_accuracy: 0.7500 - val_loss: 0.4435 - val_categorical_accuracy: 0.9000

Epoc